In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from functions import base_functions
from tqdm import tqdm
import math

In [2]:
class SymbolicLayer(nn.Module):
    def __init__(self, input_size, output_size, functions) -> None:
        super().__init__()
        self.functions = functions
        self.n_functions = len(functions)

        self.single_input_functions = [f for f in self.functions if f.n_inputs == 1]
        self.n_single = len(self.single_input_functions)

        self.double_input_functions = [f for f in self.functions if f.n_inputs == 2]
        self.n_double = len(self.double_input_functions)

        self.n_inputs_func = self.n_single + 2 * self.n_double

        weights = torch.Tensor(input_size, self.n_inputs_func)
        self.weights = nn.Parameter(weights)
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5))

        self.output = None

    def forward(self, x):

        self.output = []

        x = torch.matmul(x, self.weights)

        i_out = i_in = 0

        while i_out < self.n_single:
            self.output.append(self.single_input_functions[i_out].torch(x[i_in]))
            i_out += 1
            i_in += 1

        while i_out < self.n_functions:
            self.output.append(self.double_input_functions[i_out - self.n_single].torch(x[i_in], x[i_in + 1]))
            i_out += 1
            i_in += 2
        
        return torch.stack(self.output)

    def get_weights(self):
        return self.weights

In [3]:
class SymbolicNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, functions: list):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.functions = functions
        self.n_funcs = len(functions)
        self.output = nn.Linear(hidden_size, output_size)
        self.fc1 = SymbolicLayer(input_size, hidden_size, functions=functions)
        self.fc2 = SymbolicLayer(hidden_size, hidden_size, functions=functions)

    def forward(self, x):
        x = self.fc1(x)
        x = self.output(x)
        return x
    
    def train_n_epochs(self, x, y, epochs=100, lr=0.0001):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        for epoch in range(epochs):
            total_loss = 0 # Storing total loss during training
            # Forward pass and weight update for each sample
            for i in range(len(x)):
                optimizer.zero_grad()
                output = self.forward(x[i])
                loss = loss_fn(output, y[i])
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
        
            if epoch % 100 == 0:
                print(f'Epoch: {epoch} Loss: {loss.item()}')
    
    def get_weights(self):
        return self.fc1.get_weights(), self.fc2.get_weights(), self.output.weight


In [4]:
model = SymbolicNet(1, len(base_functions), 1, base_functions)

In [5]:
model(torch.tensor([10], dtype=torch.float32))

tensor([6.9550], grad_fn=<AddBackward0>)

In [6]:
x = torch.linspace(-math.pi, math.pi, 30).view(-1, 1)
y = torch.tensor(torch.sin(x))

model.train(True)
model.train_n_epochs(x, y, epochs=500, lr=0.01)

/tmp/ipykernel_98974/1546148179.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(torch.sin(x))


Epoch: 0 Loss: 0.3957529067993164
Epoch: 100 Loss: 0.022474229335784912
Epoch: 200 Loss: 0.02131063863635063
Epoch: 300 Loss: 0.026072992011904716
Epoch: 400 Loss: 0.021237004548311234


In [7]:
weights = model.get_weights()

In [8]:
for w, f in zip(weights[0][0], model.functions):
    print(w.item(), f.sympy())
    

0.0196909811347723 x**2
0.9432464241981506 sin(x)
